In [1]:
import sys
sys.path.append("..")

In [2]:
import os
from models.segmentation_model import SegmentationModel
import tensorflow as tf
import numpy as np

# Define parametes

In [3]:
# train_parameters = {
#     "lr": 0.001,
#     "decay_steps": 400,
#     "decay_rate": 0.95,
#     "batch_size": 16
# }

# model_parameters = {
#     "num_conv_layers": 4,
#     "conv_num_filters": 128,
#     "conv_kernel_size": [5, 9],
#     "dropout_prob": 0.05,
#     "num_bidirectional_units": 512,
#     "num_bidirectional_layers": 4,
#     "num_beams": 50
# }

# output_vocab_size= 50 # TODO: Define


# num_steps = 2

# save_energy = 2

# n_mels = 20

train_parameters = {
    "lr": 0.001,
    "decay_steps": 400,
    "decay_rate": 0.95
}

model_parameters = {
    "num_conv_layers": 2,
    "conv_num_filters": 16,
    "conv_kernel_size": [2, 2],
    "dropout_prob": 0.05,
    "num_bidirectional_units": 16,
    "num_bidirectional_layers": 2,
    "num_beams": 2
}

output_vocab_size= 50 # TODO: Define


num_steps = 2

save_energy = 2

n_mels = 20

# Train model

In [4]:
with tf.Session() as sess:
    inputs = tf.placeholder(tf.float32, [None, None, n_mels])
    targets = tf.sparse_placeholder(tf.int32)
    seq_len = tf.placeholder(tf.int32, [None])

    model = SegmentationModel(
        inputs, seq_len,
        output_vocab_size, model_parameters,
        targets, train_parameters
    )

    train_writer = tf.summary.FileWriter('../log/train_grapheme_to_phoneme_model_notebook/train', sess.graph)
    
    tf.global_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    saver = tf.train.Saver(max_to_keep=3, keep_checkpoint_every_n_hours=3)
    
    for _ in xrange(num_steps):
        
        target_indices=[[0, 0], [0, 1], [0, 2], [1, 0], [1, 1], [1, 2]]
        target_values=[1, 2, 3, 4, 5, 6]
        dense_shape=[2, 4]
        
        out = sess.run([
            model.train_op,
            model.global_step,
            model.loss,
            model.summary,
            model.training_output,
            model.prediction
        ], feed_dict={inputs: np.random.rand(2, 10, 20), seq_len: 8* np.ones(2), targets:(target_indices,target_values, dense_shape)})
        _, global_step, loss, summary, training_output, prediction  = out
        
        print global_step
        print loss
        print training_output
        print prediction

        train_writer.add_summary(summary, global_step)

        # detect gradient explosion
        if loss > 1e8 and global_step > 500:
            print('loss exploded')
            break

        if global_step % save_energy == 0 and global_step != 0:

            print('saving weights')
            if not os.path.exists('../weights/train_grapheme_to_phoneme_model_notebook/'):
                os.makedirs('../weights/train_grapheme_to_phoneme_model_notebook/')
            saver.save(sess, '../weights/train_grapheme_to_phoneme_model_notebook/', global_step=global_step)
    
    coord.request_stop()
    coord.join(threads)

1
24.7928
([SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [0, 2],
       [0, 3],
       [1, 0],
       [1, 1],
       [1, 2]]), values=array([29, 19, 19, 46, 15, 19, 42]), dense_shape=array([2, 4]))], array([[-2.50458431],
       [-3.35353732]], dtype=float32))
([SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2]]), values=array([19, 11, 19, 19, 34, 19]), dense_shape=array([2, 3]))], array([[ 5.36759949],
       [ 5.49857426]], dtype=float32))
2
24.6569
([SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1],
       [1, 2]]), values=array([42, 19, 34, 19, 24]), dense_shape=array([2, 3]))], array([[-2.87356925],
       [-3.04589844]], dtype=float32))
([SparseTensorValue(indices=array([[0, 0],
       [0, 1],
       [0, 2],
       [1, 0],
       [1, 1],
       [1, 2]]), values=array([16, 19, 34, 11, 19, 34]), dense_shape=array([2, 3]))], array([[ 5.54832935],
       [ 5.42394257